In [12]:
# will be using mnsit data set 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [13]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [14]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)
#xavier algorithm automatically determines the scale of initialization based on the number of input and output

In [15]:
mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
Z_dim = 100
h_dim = 128
#for generator 
Z = tf.placeholder(tf.float32, shape=[None, Z_dim]) #input z dims 100 vector
G_W1 = tf.Variable(xavier_init([Z_dim + y_dim, h_dim]))
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))
G_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

# For Discriminator
X = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, y_dim])
D_W1 = tf.Variable(xavier_init([X_dim + y_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))
D_W2 = tf.Variable(xavier_init([h_dim, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))
theta_D = [D_W1, D_W2, D_b1, D_b2]

Extracting ../../MNIST_data\train-images-idx3-ubyte.gz
Extracting ../../MNIST_data\train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data\t10k-labels-idx1-ubyte.gz


In [16]:
def discriminator(x, y):
    inputs = tf.concat(axis=1, values=[x, y])
    D_h1 = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)
    return D_prob, D_logit

In [17]:
def generator(z, y):
    inputs = tf.concat(axis=1, values=[z, y])
    G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

In [18]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [19]:
mb_size = 64
G_sample = generator(Z, y)
D_real, D_logit_real = discriminator(X, y)
D_fake, D_logit_fake = discriminator(G_sample, y)

D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

In [20]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out1/'):
    os.makedirs('out1/')

In [21]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
    return fig

In [ ]:
i = 0

for it in range(1000000):
    if it % 1000 == 0:
        n_sample = 16

        Z_sample = sample_Z(n_sample, Z_dim)
        y_sample = np.zeros(shape=[n_sample, y_dim])
        y_sample[:, 7] = 1

        samples = sess.run(G_sample, feed_dict={Z: Z_sample, y:y_sample})

        fig = plot(samples)
        plt.savefig('out1/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, y_mb = mnist.train.next_batch(mb_size)

    Z_sample = sample_Z(mb_size, Z_dim)
    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: Z_sample, y:y_mb})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: Z_sample, y:y_mb})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Iter: 0
D loss: 1.003
G_loss: 3.012

Iter: 1000
D loss: 0.003601
G_loss: 7.474

Iter: 2000
D loss: 0.009362
G_loss: 5.852

Iter: 3000
D loss: 0.1622
G_loss: 4.473

Iter: 4000
D loss: 0.1033
G_loss: 5.046

Iter: 5000
D loss: 0.1862
G_loss: 5.695

Iter: 6000
D loss: 0.2006
G_loss: 5.108

Iter: 7000
D loss: 0.4866
G_loss: 3.163

Iter: 8000
D loss: 0.7224
G_loss: 3.01

Iter: 9000
D loss: 0.3839
G_loss: 3.448

Iter: 10000
D loss: 0.7355
G_loss: 3.138

Iter: 11000
D loss: 0.5637
G_loss: 3.024

Iter: 12000
D loss: 0.7233
G_loss: 2.084

Iter: 13000
D loss: 0.7529
G_loss: 2.306

Iter: 14000
D loss: 0.6552
G_loss: 2.872

Iter: 15000
D loss: 0.5925
G_loss: 2.548

Iter: 16000
D loss: 0.8697
G_loss: 2.067

Iter: 17000
D loss: 0.6573
G_loss: 2.101

Iter: 18000
D loss: 1.151
G_loss: 1.651

Iter: 19000
D loss: 0.6828
G_loss: 2.24

Iter: 20000
D loss: 0.6608
G_loss: 2.26

Iter: 21000
D loss: 0.9293
G_loss: 1.742

Iter: 22000
D loss: 0.8315
G_loss: 1.49

Iter: 23000
D loss: 1.028
G_loss: 1.578

Iter: 24